In [1]:
# 필요한 패키지를 설치합니다.
# conda create -n를 통해 새로운 환경을 만들며, python 버전은 3.10.0 입니다. OS는 ubuntu 20.04입니다.
# 해당 환경을 커널로 지정합니다.
%pip install -r requirements.txt
# torch가 gpu를 인식하는 경우 아래 xheel은 pass해도 됨. 현재는 1.13.1 토치로 진행함. (gpu cuda버전마다 상이함)
#%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

  Using cached ultralytics-8.0.112-py3-none-any.whl.metadata (26 kB)
  Using cached waffle_hub-0.2.11-py3-none-any.whl.metadata (5.0 kB)
  Using cached waffle_utils-0.3.1-py3-none-any.whl.metadata (1.7 kB)
  Using cached matplotlib-3.8.1-cp310-cp310-win_amd64.whl.metadata (5.9 kB)
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached Pillow-10.1.0-cp310-cp310-win_amd64.whl.metadata (9.6 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached scipy-1.11.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached torchvision-0.16.0-cp310-cp310-win_amd64.whl.metadata (6.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached pandas-2.1.3-cp310-cp310-win_amd64.whl.metadata (18 kB)
  Using cached seaborn-0.13.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
  Using cached pydantic-1.8

In [1]:
# torch avail check
import torch
torch.cuda.is_available()

c:\Users\admin\anaconda3\envs\k_public\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
# 필요한 모듈 및 함수/클래스를 선언합니다.
import os

from waffle_hub.dataset import Dataset
from waffle_hub.hub import Hub
from waffle_utils.file import io
#from ultralytics import YOLO

os.environ['MKL_THREADING_LAYER'] = '1'

In [3]:
# 데이터셋을 불러옵니다. (데이터셋이 매우 크므로 존재한다고 가정)
# coco_file은 제공해주신 train.json의 절대경로로 지정하면 됩니다.
# coco_root_dir은 제공해주신 train 폴더를 지정해주시면 됩니다.
# Dataset 클래스의 from_coco 메서드로 dataset이라는 인스턴스를 생성하며, 학습 시 사용됩니다.

dataset = Dataset.from_coco(
    name="k_project_waffle_v1.0.0",
    task="object_detection",
    coco_file="./datasets/k_project_raw_dataset/labels/train.json",
    coco_root_dir="./datasets/k_project_raw_dataset/train/",
)

#dataset = Dataset.load(name = "k_project_waffle_v1.0.0") # 파일시스템 생성시에는 해당 클래스 매서드로 인스턴스 생성

In [ ]:
# 데이터셋을 train / val 단위로 분할합니다.
# 데이터가 너무 적어서 validate가 안됩니다. 실제 데이터의 train.json과 image가 들어있는 train폴더를 인자로 두면 정상 동작합니다.
dataset.split(train_ratio=0.995, val_ratio=0.005)

In [ ]:
# YOLOv8을 이용하므로, dataset의 format을 yolo로 export합니다.
# 실제 동작했다는 가정하에, 임의로 datasets/k_project_waffle_v1.0.0에 export폴더를 임의로 심어 두겠습니다.
dataset.export(
    "yolo"
)

In [11]:
# 배경을 따로 넣어줍니다.
# 학습에 있어서 배경의 유무는 결과값에 매우 큰 변화를 줍니다.
# 따라서 다음 코드를 이용하여 train.json 에서 제공하는 annotations 중 이용하지 않는 이미지를 train의 배경으로 넣어줍니다.

train_coco_path = './datasets/k_project_raw_dataset/labels/train.json'
train_folder = './datasets/k_project_raw_dataset/train/'

train_json = io.load_json(train_coco_path)
img_dict = {}
img_id_set = set()
ann_img_id_set = set()
ann_list = []

for anno in train_json["annotations"]:
    ann_img_id_set.add(anno["image_id"])
    ann_list.append(anno)

for index, image in enumerate(train_json["images"]):
    if not(image["id"] in ann_img_id_set) :
        src = f"{train_folder}" + image["file_name"]
        dst = "./datasets/k_project_waffle_v1.0.0/exports/YOLO/train/images/" + image["file_name"]       
        io.copy_file(src, dst)
    else :
        #print(image)
        pass

In [4]:
# 학습할 hub인스턴스를 생성하기 위한 부분입니다.
# backend부터 model_size까지 요약하자면, ultralytics yolov8 x 모델을 사용하였습니다.
hub = Hub.new(
    name="k_project_v1.0.0",
    backend="ultralytics",
    task="OBJECT_DETECTION",
    model_type="yolov8",
    model_size="x",
    categories=dataset.get_category_names()
)
#hub = Hub.load(name = "k_project_v1.0.0")      # 이미 파일시스템 생성 시 해당 load 클래스메서드로 hub 인스턴스 생성

c:\Users\admin\anaconda3\envs\k_public\lib\site-packages\waffle_hub\hub\hub.py:528: UserWarning: Super category is not specified. It may cause unexpected errors in some backends.
To avoid this warning, please specify category as a list of dictionary or Category
  warnings.warn(


In [ ]:
# 우분투에서 진행 시 올바르게 동작.
hub.train(dataset, epochs=10, batch_size=2, image_size=[640,640],workers=0)